<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Получение-данных" data-toc-modified-id="Получение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Получение данных</a></span></li><li><span><a href="#Дашборд" data-toc-modified-id="Дашборд-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Дашборд</a></span></li><li><span><a href="#Презентация" data-toc-modified-id="Презентация-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Презентация</a></span></li></ul></div>

## Получение данных

Импортируем библиотеки

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Задаем словарь параметров подключения.

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

Создаем настройку для подключения к БД.

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

In [4]:
engine = create_engine(connection_string)

Напишем SQL-запрос для получения данных из таблицы log_raw.

In [5]:
query = """
            SELECT * FROM log_raw
        """

Прочитаем и запишем полученные данные в переменную dash_visits.

In [6]:
dash_visits = pd.io.sql.read_sql(query, con = engine)

Выведем на экран первые 5 строк датафрейма.

In [7]:
dash_visits.head()

,event_id,age_segment,event,item_id,item_topic,item_type,source_id,source_topic,source_type,ts,user_id
0,2931262,18-25,show,1793260,Красота,native,8388799,История,publisher,1569349657638,2185026
1,2931277,18-25,show,7465641,Культура,native,5837959,История,publisher,1569349657971,6845238
2,2931285,18-25,show,406073,История,url,2335872,Сад и дача,domain,1569349658408,5244363
3,2931247,26-30,show,752665,Путешествия,native,6963489,Здоровье,publisher,1569349657189,7553074
4,2931278,26-30,click,6843081,Юмор,url,5427748,Семейные отношения,domain,1569349658054,1317585


Преобразуем данные в колонке `ts` в тип datetime64[ms].

In [8]:
dash_visits['ts'] = dash_visits['ts'].astype(dtype='datetime64[ms]')

Изучим результат. 

In [9]:
dash_visits.head()

,event_id,age_segment,event,item_id,item_topic,item_type,source_id,source_topic,source_type,ts,user_id
0,2931262,18-25,show,1793260,Красота,native,8388799,История,publisher,2019-09-24 18:27:37.638,2185026
1,2931277,18-25,show,7465641,Культура,native,5837959,История,publisher,2019-09-24 18:27:37.971,6845238
2,2931285,18-25,show,406073,История,url,2335872,Сад и дача,domain,2019-09-24 18:27:38.408,5244363
3,2931247,26-30,show,752665,Путешествия,native,6963489,Здоровье,publisher,2019-09-24 18:27:37.189,7553074
4,2931278,26-30,click,6843081,Юмор,url,5427748,Семейные отношения,domain,2019-09-24 18:27:38.054,1317585


Проанализируем датасет на пропуски в данных.

In [10]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322391 entries, 0 to 322390
Data columns (total 11 columns):
event_id        322391 non-null int64
age_segment     322391 non-null object
event           322391 non-null object
item_id         322391 non-null int64
item_topic      322391 non-null object
item_type       322391 non-null object
source_id       322391 non-null int64
source_topic    322391 non-null object
source_type     322391 non-null object
ts              322391 non-null datetime64[ns]
user_id         322391 non-null int64
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 27.1+ MB


Пропуски отсутсивуют.  

Проверим на наличие дубликатов в данных.

In [11]:
dash_visits[dash_visits.duplicated() == True]

,event_id,age_segment,event,item_id,item_topic,item_type,source_id,source_topic,source_type,ts,user_id


Дубликаты также отсутвуют.  

Выведем на экран уникальные значения в столбцах 'age_segment', 'event', 'item_topic', 'item_type', 'source_topic', 'source_type' и посмотрим нет ли неявных дубликатов.

In [12]:
for column in ['age_segment', 'event', 'item_topic', 'item_type', 'source_topic', 'source_type']:
    print(dash_visits[column].unique(), '\n')

['18-25' '26-30' '31-35' '36-40' '45+' '41-45'] 

['show' 'click' 'view'] 

['Красота' 'Культура' 'История' 'Путешествия' 'Юмор' 'Наука' 'Дети'
 'Полезные советы' 'Шоу' 'Семья' 'Деньги' 'Психология' 'Россия'
 'Искусство' 'Отношения' 'Женская психология' 'Здоровье'
 'Интересные факты' 'Общество' 'Подборки' 'Рассказы' 'Скандалы' 'Женщины'
 'Знаменитости' 'Туризм'] 

['native' 'url' 'narrative_card' 'post'] 

['История' 'Сад и дача' 'Здоровье' 'Семейные отношения' 'Знаменитости'
 'Россия' 'Путешествия' 'Кино' 'Полезные советы' 'Психология' 'Спорт'
 'Строительство' 'Одежда' 'Искусство' 'Финансы' 'Ремонт' 'Семья' 'Дети'
 'Технологии' 'Авто' 'Сделай сам' 'Интерьеры' 'Деньги' 'Музыка' 'Еда'
 'Политика'] 

['publisher' 'domain'] 



С данными все в порядке.  

Создадим переменную path, в которой укажем локальный путь для сохранения scv-файла.

In [13]:
path = '/Users/segagoose/Documents/YandexPracticum/Автоматизация/проект'

Сохраним датасет dash_visits в файл dash_visits.csv.

In [14]:
dash_visits.to_csv(path + '/dash_visits.csv')

Можно переходить к построению дашборда.

## Дашборд 

Ознакомиться с дашбордом можно [здесь](https://public.tableau.com/app/profile/serhio/viz/project_16527135461830/Dashboard1?publish=yes).

## Презентация

Ознакомиться с презентацией можно [здесь](https://disk.yandex.ru/i/XdzIXTWZxNUerA).